## 04 - apply to all

### 1 - load model

In [1]:
from bisenet import engine
import config
from config import *
from utils import *
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [38]:
def predict_batch(model, batch, device='cpu'):
    """[summary]

    Args:
        model ([model]): [deeplab model]
        img ([list]): [batch of images]

    Returns:
        [type]: [description]
    """    
    model.eval()
    model.to(device)
    with torch.no_grad():
        
        batch = batch.to(device)
        # print(img)
        pred, *_= model(batch)
        # print(pred.shape)
        masks = pred.permute(0, 2, 3, 1).detach().cpu().numpy()
        masks = masks.argmax(3).reshape(-1, 128, 128, 1)
        return np.array(masks, dtype=np.uint8)

In [3]:
bisenet_engine = engine.bisenet_engine(config)

In [4]:
model = bisenet_engine.model
model_weight = torch.load(config.model_weight)
model_weight = model_weight["model_state_dict"]
model.load_state_dict(model_weight)

<All keys matched successfully>

### 2 - Load InfoMAP and prepare batch dataset

In [5]:
info_map = idp.jsonfile.read_json(r"Z:\hwang_Pro\data\2022_tanashi_broccoli\12_head_segment\broccoli_autumn21-20220405_0.json")

In [12]:
import imageio as io
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
        A.Resize(128, 128),
        # A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        ToTensorV2(),
    ])

In [36]:
values = info_map['2_DJI_0355']
croped_image_path = values['cropedImagePath']
print(croped_image_path)
coords = values['headCoordOnCroppedImage']
base = 75
img = io.imread(croped_image_path)
h, w, _ = img.shape
one_mask = np.zeros((h, w, 1), dtype=np.uint8)

# convert to numpy array
points = np.array(coords, dtype=np.int32)
# calcualte x0, y0, x1, y1
y0 = points[:, 1] - base
x0 = points[:, 0] - base
y1 = points[:, 1] + base
x1 = points[:, 0] + base
# avoid minus number
y0[y0 < 0] = 0
x0[x0 < 0] = 0
y1[y1 > h] = h
x1[x1 > w] = w

bboxs = np.array(list(zip(y0, x0, y1, x1)))
# get sub images
batch = []
for idx, (y_0, x_0, y_1, x_1) in enumerate(bboxs):
    sub_img = img[y_0:y_1, x_0:x_1, :3]
    sub_img = transform(image=sub_img)['image']
    batch.append(sub_img)

batch = torch.stack(batch)

Z:/hwang_Pro/data/2022_tanashi_broccoli/12_head_segment/broccoli_autumn21-20220405_0/2_DJI_0355.png


### 3- Model Inferences

In [39]:
rst = predict_batch(model, batch, device)

In [41]:
from torchvision import transforms
for box, mask in zip(bboxs, rst):
    # print(np.unique(mask))
    y0, x0, y1, x1 = box
    # print(mask, mask.shape) 
    mask = transforms.Resize((y1-y0, x1-x0))(torch.tensor(mask).permute((2,0,1)))*255
    
    mask[mask > 50] = 255
    mask[mask <= 50] = 100
    # print('masks:', np.unique(mask))
    mask = np.asarray(mask).transpose((1,2,0))
    
    # print(mask.shape)
    one_mask[y0:y1, x0:x1, :] = mask

    # save visualization
    io.imsave('./test.png', one_mask)